# When Things Go Bad: Common Errors and Solutions

A few issues and how to deal with them.

## Time

* Remember that time is UTC, not your local time.

```bash
mtavares@f1ccff851ada:~/work$ blazar lease-update --start-date "2019-03-19 14:29" --end-date "2019-03-19 14:30" mtavares-haswell-test
ERROR: Cannot modify the start date of already started leases
mtavares@f1ccff851ada:~/work$ blazar lease-update --end-date "2019-03-19 15:46" mtavares-haswell-testERROR: End date must be later than current and start date
mtavares@f1ccff851ada:~/work$ date
Tue Mar 19 18:30:01 UTC 2019
mtavares@f1ccff851ada:~/work$ blazar lease-update --end-date "2019-03-19 18:31" mtavares-haswell-test
Updated lease: mtavares-haswell-test
mtavares@f1ccff851ada:~/work$
```


## A terminated lease is still an existing lease

A lease can assume different states such as:
* PENDING
* ACTIVE
* TERMINATING
* TERMINATED
* ERROR

A description of what each state stands for is available in the [blazar docs][1].

When a lease is `TERMINATED`, it does not go away:

```bash
mtavares@f1ccff851ada:~/work$ blazar lease-list
+--------------------------------------+-----------------------+----------------------------+----------------------------+
| id                                   | name                  | start_date                 | end_date                   |
+--------------------------------------+-----------------------+----------------------------+----------------------------+
| 1cb87f11-882c-42c9-b7b7-887de08005a2 | mtavares-haswell-test | 2019-03-22T19:36:00.000000 | 2019-03-23T19:36:00.000000 |
+--------------------------------------+-----------------------+----------------------------+----------------------------+
mtavares@f1ccff851ada:~/work$ date
Mon Mar 25 03:33:17 UTC 2019
mtavares@f1ccff851ada:~/work$
```

If you just create a lease after one is terminated, it will not overwrite the
old one.
```bash
mtavares@f1ccff851ada:~/work$ blazar lease-list
+--------------------------------------+-----------------------+----------------------------+----------------------------+
| id                                   | name                  | start_date                 | end_date                   |
+--------------------------------------+-----------------------+----------------------------+----------------------------+
| 447ee5ab-80b4-4c17-bf99-b6750c130c9b | mtavares-haswell-test | 2019-03-19T18:36:00.000000 | 2019-03-20T18:36:00.000000 |
| e710063e-f7e7-49d1-90ce-0f2b2e41e3eb | mtavares-haswell-test | 2019-03-19T15:45:00.000000 | 2019-03-19T18:31:00.000000 |
+--------------------------------------+-----------------------+----------------------------+----------------------------+
mtavares@f1ccff851ada:~/work$
```

We will need to delete it so it release the resources, One would expect that `blazar lease-delete mtavares-haswell-test` would be all we need to do to delete the lease and release all the resources tied down to it, but that is not the case:

1. If there are two leases with the same name, we cannot get info on a given lease by just entering its name.

```bash
mtavares@f1ccff851ada:~/work$ blazar lease-show  mtavares-haswell-test
There are more than one appropriate resources for the name 'mtavares-haswell-test' and type 'lease'
mtavares@f1ccff851ada:~/work$
```

Much less deleting it. The way to delete it is using the unique id (first column in the previous table).

2. The resources -- servers, externally facing IPs, etc -- associated with a lease are not freed when lease is gone. 

[1]: https://docs.openstack.org/blazar/latest/user/state-machines.html

## Deleting a lease does NOT release all the resources attached to it

The resources -- servers, externally facing IPs, etc -- associated with a lease are not freed when lease is gone. That is probably fine with an externally facing IP, and maybe a server that is already running can be fed into a new lease or something, but what if this server has been terminated already?


## When getting a Status = ERROR, look at the 'fault' for clues
For instance, let's say some of the nodes you were trying to create returned that error:

```bash
openstack server list
+--------------------------------------+----------------------+--------+--------------------------------------------------+----------------+-----------+
| ID                                   | Name                 | Status | Networks                                         | Image          | Flavor    |
+--------------------------------------+----------------------+--------+--------------------------------------------------+----------------+-----------+
| 55c049cc-e013-42db-b371-b3249d06ac23 | my-first-server      | ERROR  |                                                  | CC-Ubuntu18.04 | baremetal |
+--------------------------------------+----------------------+--------+--------------------------------------------------+----------------+-----------+
```
You should then ask what is up by looking at the `fault` field for that server:
```bash
mtavares@f1ccff851ada:~/work$ openstack server show --format value -c fault my-first-server
{'message': 'No valid host was found. There are not enough hosts available.', 'code': 500, 'created': '2019-03-20T14:22:38Z'}
mtavares@f1ccff851ada:~/work$
```

So, `fault` tell us that `No valid host was found. There are not enough hosts available`. Do not expect these messages to be helpful at all time; might just give a generic error string which you will need to know the context and look into a document like this for clues. Incidentally, we will talk about this very error message later on.

NOTE: `fault` is case sensitive, and so is `status`. It is `Status` when you do `blazar lease-list` or `openshift server list` but `status` when you run `blazar lease-show` or `openshift server show`. THis is important when you are trying to filter for the message.